<a href="https://colab.research.google.com/github/hardeybisey/neural-network/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# !pip install tokenizers
# !pip install datasets
# !pip install transformers

In [ ]:
from tokenizers import CharBPETokenizer

# Read the dataset

In [ ]:
corpus = open('input.txt', 'r', encoding='utf-8').read()
print("length of dataset in characters: ", len(corpus))
print(f"{corpus[:200]}")

length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


# Build a custom tokenizer with hugging face  tokenizer API

In [ ]:
tokenizer = CharBPETokenizer()
tokenizer.train_from_iterator(corpus)

In [ ]:
# testing the tokenizer
encoded = tokenizer.encode(corpus[:50])
print(encoded.tokens)
print(tokenizer.decode(encoded.ids))

['F', 'i', 'r', 's', 't</w>', 'C', 'i', 't', 'i', 'z', 'e', 'n</w>', ':</w>', 'B', 'e', 'f', 'o', 'r', 'e</w>', 'w', 'e</w>', 'p', 'r', 'o', 'c', 'e', 'e', 'd</w>', 'a', 'n', 'y</w>', 'f', 'u', 'r', 't', 'h', 'e', 'r</w>', ',</w>', 'h', 'e', 'a', 'r</w>']
First Citizen : Before we proceed any further , hear


In [ ]:
from tokenizers import Tokenizer, models, normalizers , Regex, pre_tokenizers, trainers,processors,decoders

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
tokenizer.normalizer = normalizers.Sequence([
                  normalizers.Replace(Regex(r"[\p{other}&&[^\n\t\r]]"), ""), normalizers.Replace(Regex(r"[\s]"), " "),
                  # normalizers.Lowercase(),
                  normalizers.NFD(), normalizers.StripAccents()])
tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()])
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "MASK"]
trainer = trainers.WordPieceTrainer(vocab_size=2000, special_tokens=special_tokens)

tokenizer.train_from_iterator(corpus , trainer=trainer)

cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
tokenizer.post_processor = processors.TemplateProcessing(
    single = f"[CLS]:0 $A:0 [SEP]:0",
    pair = f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens = [("[CLS]", cls_token_id), ("[SEP]",sep_token_id)])
tokenizer.decoder = decoders.WordPiece(prefix="##")
tokenizer.encode(corpus[:50]).tokens


# worked
# tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
# tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)
# tokenizer.normalizer = normalizers.Sequence(
#     [normalizers.NFD(), normalizers.StripAccents()])
# tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Split('', 'isolated'),
#                                                    pre_tokenizers.Punctuation()])

# tokenizer.pre_tokenizer.pre_tokenize_str("Let's test my \n pre-tokenizer.")
# special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
# trainer = trainers.WordPieceTrainer(vocab_size=5000, special_tokens=special_tokens)
# tokenizer.train_from_iterator(text, trainer=trainer)

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
raw_datasets['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [ ]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [ ]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    for word in word_ids:
        if word == None:
            new_labels.append(-100)
        else:
            new_labels.append(labels[word])
    return new_labels


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    old_labels = examples["ner_tags"]
    new_labels = []
    for i , labels in enumerate(old_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(old_labels[i],word_ids))
    tokenized_inputs["labels"] = new_labels 
    return tokenized_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_and_align_labels,batched=True, remove_columns=raw_datasets["train"].column_names)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
#adding paddiing at bacth level
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [ ]:
# !pip install seqeval evaluate

Metrics

In [ ]:
import evaluate

metric = evaluate.load("seqeval")

In [ ]:
# customizing the metric output
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

Cloning https://huggingface.co/hardeybisey/bert-finetuned-ner into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
